In [1]:
import os
os.getcwd()

'/Users/apple/Deep Learning and Neural Networks/Distracted Driver'

In [2]:
import cv2 as cv

In [3]:
train_dir="/Users/apple/Deep Learning and Neural Networks/Distracted Driver"+"/state-farm-distracted-driver-detection/imgs/train"
train_dir
test_dir="/Users/apple/Deep Learning and Neural Networks/Distracted Driver"+"/state-farm-distracted-driver-detection/imgs/test"

In [4]:
image=train_dir+"/0"+"/img_100026.jpg"
image1=cv.imread(image)
gray=cv.cvtColor(image1,cv.COLOR_BGR2GRAY)


In [5]:
image1.shape

(480, 640, 3)

In [6]:
import matplotlib.pyplot as plt
cv.imshow("ex",image1)
cv.waitKey(0)

100

In [19]:
blur=cv.GaussianBlur(image1,(3,3),cv.BORDER_DEFAULT)
cv.imshow("blur",blur)
cv.waitKey(0)

100

In [42]:
cv.destroyAllWindows()

In [21]:
canny=cv.Canny(image1,125,175)
canny1=cv.Canny(blur,125,175)
cv.imshow("canny",canny)
cv.waitKey(0)

100

In [23]:
dilated=cv.dilate(canny,(3,3),iterations=1)
cv.imshow("dilated",dilated)
cv.waitKey(0)

100

In [24]:
eroded=cv.erode(dilated,(3,3),iterations=1)
cv.imshow("eroded",eroded)
cv.waitKey(0)

100

In [43]:
#Contours

In [26]:
ret,thresh=cv.threshold(canny,125,255,cv.THRESH_BINARY)

In [27]:
cv.imshow("thresh",thresh)
cv.waitKey(0)
cv.destroyAllWindows()

In [28]:
contours, hierarchies=cv.findContours(thresh,cv.RETR_LIST,cv.CHAIN_APPROX_NONE)
print(f'{len(contours)} contours(s) found?')

835 contours(s) found?


In [67]:
import numpy as np
blank=np.zeros(image1.shape,dtype='uint8')
cv.drawContours(blank,contours,-1,(0,0,255),2)
cv.imshow("contours drawn",blank)
cv.waitKey(0)
cv.destroyAllWindows()

In [61]:
#Bilateral Blur

In [62]:
bilateral=cv.bilateralFilter(image1,5,15,15)
cv.imshow("bilateral",bilateral)
cv.waitKey(0)
cv.destroyAllWindows()

In [63]:
canny2=cv.Canny(bilateral,125,175)

In [6]:
#Thresholding

In [7]:
gray=cv.cvtColor(image1,cv.COLOR_BGR2GRAY)
cv.imshow("simple threshold",gray)
cv.waitKey(0)

100

In [8]:
#inverse threshold

In [9]:
threshold,thresh_inv=cv.threshold(gray,150,255,cv.THRESH_BINARY_INV)
cv.imshow("simple_threshold_inv",thresh_inv)
cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
#adaptive threshold

In [11]:
adaptive_thresh=cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,11,3)
cv.imshow("adaptive",adaptive_thresh)
cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
#gradients and edge detection

In [14]:
import numpy as np
gray=cv.cvtColor(image1,cv.COLOR_BGR2GRAY)
lap=cv.Laplacian(gray,cv.CV_64F)
lap=np.uint8(np.absolute(lap))
cv.imshow("laplacian",lap)
cv.waitKey(0)
cv.destroyAllWindows()

In [15]:
#sobel

In [16]:
sobelx=cv.Sobel(gray,cv.CV_64F,1,0)
sobely=cv.Sobel(gray,cv.CV_64F,0,1)
combined_sobel=cv.bitwise_or(sobelx,sobely)

In [17]:
cv.imshow("sobelx",sobelx)
cv.imshow("sobely",sobely)
cv.imshow("sobel_combined",combined_sobel)
cv.waitKey(0)
cv.destroyAllWindows()

In [34]:
#data preprocessing

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen=ImageDataGenerator(rescale=1.0/255,validation_split=0.2)
validation_data_gen=ImageDataGenerator(rescale=1.0/255,validation_split=0.2)
test_data_gen=ImageDataGenerator(rescale=1.0/255)

In [7]:
import tensorflow as tf

In [9]:
train_data=tf.keras.utils.image_dataset_from_directory(train_dir,
                                                      validation_split=0.2,
                                                      subset="training",
                                                      image_size=(100,100),
                                                       seed=123,
                                                      batch_size=128,
                                                      label_mode="categorical")
validation_data=tf.keras.utils.image_dataset_from_directory(train_dir,
                                                           validation_split=0.2,
                                                           subset="validation",
                                                           image_size=(100,100),
                                                            seed=123,
                                                          batch_size=128,
                                                          label_mode="categorical")

Found 22424 files belonging to 11 classes.
Using 17940 files for training.
Found 22424 files belonging to 11 classes.
Using 4484 files for validation.


In [13]:
training_generator=train_data_gen.flow_from_directory(train_dir,
                                                     target_size=(100,100),
                                                     batch_size=128,
                                                     class_mode="categorical",
                                                     subset="training")
validation_generator=validation_data_gen.flow_from_directory(train_dir,
                                                            target_size=(100,100),
                                                            batch_size=128,
                                                            class_mode="categorical",
                                                            subset="validation")

Found 17943 images belonging to 11 classes.
Found 4481 images belonging to 11 classes.


In [14]:
from keras import layers
from keras import models


In [15]:
model=models.Sequential()
model.add(layers.Conv2D(64,(3,3),activation="relu",input_shape=(100,100,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())

model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(11,activation="softmax"))

In [16]:
from keras import optimizers
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),loss="categorical_crossentropy",metrics=["acc"])

In [ ]:
history=model.fit(training_generator,
                 #steps_per_epoch=230,
                 epochs=10,
                 validation_data=validation_generator)
                 #validation_steps=45)

Epoch 1/10
141/141 [==============================] - ETA: 0s - loss: 1.3370 - acc: 0.5643